# <center> **Практическая работа 1. Отбор и селекция признаков**

### Условие задачи:

Необходимо обучить модель линейной регрессии на найденных двумя способами трёх важных признаках и сравнить полученные результаты.

### Импортируем библиотеки

In [1]:
# Для работы с датасетом
import pandas as pd
import numpy as np

# Для работы с признаками и построения моделей
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

### Загружаем данные

In [115]:
# Загружаем DataFrame
ford_price_data = pd.read_excel('Прочие файлы/data_ford_price.xlsx')

# Создаём копию данных
ford_price_df = ford_price_data.copy()

In [ ]:
# Выводим первые пять строк
ford_price_df.head()

## <center> Предобработка данных

### Обработка пропусков

Так как необходимо предсказать цену автомобиля. Для обучения модели выделим целевой столбец:

In [116]:
X = ford_price_df.drop(columns = 'price')
y = ford_price_df['price']

Разделим выборку на тренировочную и тестовую в соотношении 80/20:

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 30)

### Обработка пропусков с помощью ML

In [ ]:
# Предскажем пропущенные значения в признаке 'weather' с помощью ML
data = X.copy()
 
test_data = data[data['weather'].isnull()]
data.dropna(inplace=True)
 
y_train = data['weather']
X_train = data.drop(['size','weather','drive'], axis=1)
X_test = test_data.drop(['size','weather','drive'], axis=1)
 
one_hot_encoder = OneHotEncoder()
categorial_cols = ['cylinders', 'title_status', 'transmission']
 
X_train_onehot = one_hot_encoder.fit_transform(X_train[categorial_cols]).toarray()
X_test_onehot = one_hot_encoder.transform(X_test[categorial_cols]).toarray()
 
columns = one_hot_encoder.get_feature_names_out(categorial_cols)
X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)
 
X_train = X_train.reset_index().drop(['index'], axis = 1)
X_test = X_test.reset_index().drop(['index'], axis = 1)
y_train = y_train.reset_index().drop(['index'], axis = 1)
 
X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)
 
X_train_new = X_train_new.drop(columns=categorial_cols)
X_test_new = X_test_new.drop(columns=categorial_cols)
 
 
model = LinearRegression()
model.fit(X_train_new, y_train)
 
y_pred = model.predict(X_test_new)

# Вставим найденную замену на место пропусков в столбце weather
for i, ni in enumerate(test_data.index[:len(X)]):
    X['weather'].loc[ni] = y_pred[i]

In [ ]:
# Предскажем пропущенные значения в признаке 'size' с помощью ML
data = X.copy()
 
test_data = data[data['size'].isnull()]
data.dropna(inplace=True)
 
y_train = data['size']
X_train = data.drop(['size','drive'], axis=1)
X_test = test_data.drop(['size', 'drive'], axis=1)
 
one_hot_encoder = OneHotEncoder()
categorial_cols = ['cylinders', 'title_status', 'transmission']
 
X_train_onehot = one_hot_encoder.fit_transform(X_train[categorial_cols]).toarray()
X_test_onehot = one_hot_encoder.transform(X_test[categorial_cols]).toarray()
 
columns = one_hot_encoder.get_feature_names_out(categorial_cols)
X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)
 
X_train = X_train.reset_index().drop(['index'], axis = 1)
X_test = X_test.reset_index().drop(['index'], axis = 1)
y_train = y_train.reset_index().drop(['index'], axis = 1)
 
X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)
 
X_train_new = X_train_new.drop(columns=categorial_cols)
X_test_new = X_test_new.drop(columns=categorial_cols)
 
# Для заполения пропусков в признаке 'size' понадобиться классификатор, так как он является категориальным 
model = LogisticRegression()
model.fit(X_train_new, y_train)
 
y_pred = model.predict(X_test_new)

# Вставим найденную замену на место пропусков в столбце size
for i, ni in enumerate(test_data.index[:len(X)]):
    X['size'].loc[ni] = y_pred[i]

In [ ]:
# Предскажем пропущенные значения в признаке 'drive' с помощью ML
data = X.copy()
 
test_data = data[data['drive'].isnull()]
data.dropna(inplace=True)
 
y_train = data['drive']
X_train = data.drop(['size','drive'], axis=1)
X_test = test_data.drop(['size', 'drive'], axis=1)
 
one_hot_encoder = OneHotEncoder()
categorial_cols = ['cylinders', 'title_status', 'transmission']
 
X_train_onehot = one_hot_encoder.fit_transform(X_train[categorial_cols]).toarray()
X_test_onehot = one_hot_encoder.transform(X_test[categorial_cols]).toarray()
 
columns = one_hot_encoder.get_feature_names_out(categorial_cols)
X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)
 
X_train = X_train.reset_index().drop(['index'], axis = 1)
X_test = X_test.reset_index().drop(['index'], axis = 1)
y_train = y_train.reset_index().drop(['index'], axis = 1)
 
X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)
 
X_train_new = X_train_new.drop(columns=categorial_cols)
X_test_new = X_test_new.drop(columns=categorial_cols)
 
# Для заполения пропусков в признаке 'drive' понадобиться классификатор, так как он является категориальным 
model = LogisticRegression()
model.fit(X_train_new, y_train)
 
y_pred = model.predict(X_test_new)

# Вставим найденную замену на место пропусков в столбце size
for i, ni in enumerate(test_data.index[:len(X)]):
    X['drive'].loc[ni] = y_pred[i]

### Кодирование признаков

Создаём список номинальных признаков

In [ ]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']

for column in columns_to_change:
    print('Число уникальных значений признака {}: '.format(column), X[column].nunique())

In [122]:
# Создаём список со столбцами для кодировки
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']

# Создаём экземпляр класса OneHotEncoder()
encoder = OneHotEncoder()

# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_encoder = encoder.fit_transform(X[columns_to_change]).toarray()

# Запишем полученные названия новых колонок в отдельную переменную
column_names = encoder.get_feature_names_out(columns_to_change)

# Преобразовываем полученный массив закодированных данных в формат DataFrame
data_encoder = pd.DataFrame(data=data_encoder, index=X.index, columns=column_names)

# Объединяем оба датасета
X = pd.concat([X, data_encoder], axis=1)

# Удаляем закодированные столбцы columns_to_change из полученной таблицы
X = X.drop(columns = columns_to_change)

### Работа с выбросами

In [ ]:
# Ищем выбросы в обучающей выборке
iso = IsolationForest(contamination = 0.1)
y_predicted = iso.fit_predict(X)
 
# Выберем все строки, которые не являются выбросами
mask = y_predicted != -1
X = X[mask]

In [124]:
# Удаляем ненужные строки в векторе правильных ответов
y = y.iloc[X.index]

### Масштабирование признаков

In [125]:
# Инициализируем стандартизатор StandardScaler()
scaler = StandardScaler()

col_names = X.columns
x = X[col_names]
X[col_names] = scaler.fit_transform(x.values)

### Отбор признаков

Разделим выборку на тренировочную и тестовую в соотношении 80/20:

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 40)

Построим тепловую карту корреляции признаков

In [ ]:
plt.rcParams['figure.figsize'] = (15, 15)
sns.heatmap(X_train.corr(), annot=True, fmt='.1f');

Создадим 2 копии матрицы наблюдения X 

In [128]:
X_1 = X.copy()
X_2 = X.copy()

**МЕТОД РЕКУРСИВНОГО ИСКЛЮЧЕНИЯ ПРИЗНАКОВ**

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size=0.3, random_state=40)

In [ ]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select = 3, step = 1)
selector = selector.fit(X_train, y_train)
selector.get_feature_names_out()

In [ ]:
# Выводим названия признаков
X_train.columns

In [ ]:
# Ранжировка признаков
selector.ranking_

In [132]:
# Удаляем наименее важные признаки
X_1 = X_1.drop(columns=['year', 'condition', 'odometer', 'lat', 'long', 'weather',
       'cylinders_3', 'cylinders_4', 'cylinders_5', 'cylinders_6',
       'cylinders_8', 'cylinders_10', 'title_status_clean',
       'title_status_lien', 'title_status_missing', 'title_status_rebuilt',
       'title_status_salvage', 'transmission_automatic', 'transmission_manual',
       'transmission_other', 'size_compact', 'size_full-size', 'size_mid-size', 'size_sub-compact'], axis=1)

In [135]:
# Делим выборку
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size=0.3, random_state=40)

In [ ]:
# Обучаем модель линейной регрессии
model = LinearRegression()
model.fit(X_train,  y_train)
y_predicted = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

**МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ**

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X_2, y, test_size=0.3, random_state=40)

In [ ]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

In [ ]:
# Выводим названия признаков
X_train.columns

In [140]:
# Удаляем наименее важные признаки
X_2 = X_2.drop(columns=['lat', 'long', 'weather',
       'cylinders_3', 'cylinders_4', 'cylinders_5', 'cylinders_6',
       'cylinders_8', 'cylinders_10', 'title_status_clean',
       'title_status_lien', 'title_status_missing', 'title_status_rebuilt',
       'title_status_salvage', 'transmission_automatic', 'transmission_manual',
       'transmission_other', 'drive_4wd', 'drive_fwd', 'drive_rwd',
       'size_compact', 'size_full-size', 'size_mid-size', 'size_sub-compact'], axis=1)

In [141]:
# Делим выборку
X_train, X_test, y_train, y_test = train_test_split(X_2, y, test_size=0.3, random_state=40)

In [ ]:
# Обучаем модель линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

## <center> Вывод

В ходе предобработки данных и выделения 3-х наиболее значимых признаков методами *RFE* и *SelectKBest* и обучения модели линейной регрессии на каждом из этих столбцов были получены следующие показатели: 

- метод *RFE* наиболее важными выявил признаки ['drive_4wd', 'drive_fwd', 'drive_rwd'], показатель *MAE* составил *MAE: 8356.863*; 
- метод *SelectKBest* наиболее важными выявил признаки ['year', 'condition', 'odometer'], показатель *MAE* составил *MAE: 4617.003*.

По данным показателям можно сделать вывод, что наилучшее качество модели было достигнуто с помощью призннаков, выбраннх методом *SelectKBest*: показатель *MAE* оказался практически вдвое ниже чем у модели, обученной на отобранных признаках методом *RFE*.